In [128]:
# data analysis and clean
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

# paramter select.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict

# Learning curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

In [129]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
combine = [train_df, test_df]

DataFrame 是Series构成的字典dict. Series是DataFrame的值。 columns 是dict的键.
而index 是一个 可用标签.
内部value 是直接组织成ndarray的.

Getting 访问 -- []
    1 索引方式 df[key]        
        索引的是字典键key. 也就是只能索引列, 并且不能使用 数字位置索引 必须用键索引 
    2 切片方式 df[start:end]  
        只能切片行, 可以使用位置索引(数字), 也可以使用label(行标)

label访问, -- loc[] label 必须在 index_list中.
    1 loc[label(行标签), key_list]   label 必须是行标签, 不能是数字位置
        In [26]: df.loc['20130102']
        Out[26]: 
        A    0.469112
        B   -0.282863
        C   -1.509059
        D   -1.135632
        Name: 2013-01-01 00:00:00, dtype: float64
    
    2 loc[start:end, key_list]  start end 可以是行标签, 也可以是数字位置
        In [27]: df.loc[:,['A','B']]
        Out[27]: 
                           A         B
        2013-01-01  0.469112 -0.282863
        2013-01-02  1.212112 -0.173215
        2013-01-03 -0.861849 -2.104569
        2013-01-04  0.721555 -0.706771
        2013-01-05 -0.424972  0.567020
        2013-01-06 -0.673690  0.113648
    3 loc[list_True-False, key_list]
        list_True-False 经常是一个判断结果list  _boolean访问
        df.loc[ df['Age'] <= 16.0, 'Age'] = 0
        
postion访问 -- iloc[]
    1 iloc[] 数字位置索引, 完全同于ndarray, 返回一个DataFrame, 或者一个Series, 或者直接一个值.
        In [32]: df.iloc[3]
        Out[32]: 
        A    0.721555
        B   -0.706771
        C   -1.039575
        D    0.271860
        Name: 2013-01-04 00:00:00, dtype: float64

        In [33]: df.iloc[3:5,0:2]
        Out[33]: 
                           A         B
        2013-01-04  0.721555 -0.706771
        2013-01-05 -0.424972  0.567020
        
        In [37]: df.iloc[1,1]
        Out[37]: -0.17321464905330858
boolean访问 -- [boolean_index]  boolean_index 判断条件
        Using a single column’s values to select data.
        In [39]: df[df.A > 0]
        Out[39]: 
                           A         B         C         D
        2013-01-01  0.469112 -0.282863 -1.509059 -1.135632
        2013-01-02  1.212112 -0.173215  0.119209 -1.044236
        2013-01-04  0.721555 -0.706771 -1.039575  0.271860
        
        In [40]: df[df > 0]
        Out[40]: 
                           A         B         C         D
        2013-01-01  0.469112       NaN       NaN       NaN
        2013-01-02  1.212112       NaN  0.119209       NaN
        2013-01-03       NaN       NaN       NaN  1.071804
        2013-01-04  0.721555       NaN       NaN  0.271860
        2013-01-05       NaN  0.567020  0.276232       NaN
        2013-01-06       NaN  0.113648       NaN  0.524988

推荐访问方式: loc, iloc ,iat[index, column]访问某个位置.


Missing Data
    不明白为什么要搞反了, 可能是因为, DataFrame 是一个 Series的dict, 所以dict是更高的维度, 而Series内部的元素是最低维度.
    axis = 0 代表的是index 行
    axis = 1 代表的是key 列, 一般drop都是去掉缺失值太多的 feature. 所以一般用到的都是 drop(
    
1 df.dropna(axis=0)
2 df.drop(key_list, axis=0)
3 df.fillna(value), Series.fillna(value)

Operations


1 Pclass 有很重要的相关性.
QI 经常使用 groupby 对某个feature分组 查看目标label值方式查看 feature 与 label 相关关系
1 DataFrame [ list ]  直接索引的是 dict方式的. 对列进行 访问
2 DataFrame .groupby ['key'] 按 key值不同 进行分组, 得到分组对象pandas.core.groupby.DataFrameGroupBy, 应该是一个临时视图
3 DataFrame .sort_values(by='key')  按key排序

In [130]:
pclass_group = train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False)
pclass_group_survived_mean = pclass_group.mean().sort_values(by='Survived', ascending=False)
print(type(pclass_group.mean()))
print(pclass_group_survived_mean)

<class 'pandas.core.frame.DataFrame'>
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


2 Cabin, a 缺失了太多数值, 可以直接忽略不计, 可能稍微有些作用 填充缺失值.
1 Series.fillna() 填充缺失值
2 Series.str.extract() 从str中抽取 正则表达式 代表的str
3 Series.map函数 通过一个函数, 或者 dict. 进行映射新值
4 Series.astype() 修改Series的数据类型

In [131]:
cabin_map = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'T': 0, 'U': 1}
for df in combine:
    df['Cabin'] = df['Cabin'].fillna('U')
    df['Cabin'] = df.Cabin.str.extract('([A-Za-z])', expand=False)
    df['Cabin'] = df['Cabin'].map(cabin_map).astype(int)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,1,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,1,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,0,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,1,S


3 Ticket drop. 
? 1 DataFrame.drop() paramter -- ['Ticket'] axis = 1 为什么用axis = 1呢

In [132]:
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)
combine = [train_df, test_df]
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,1,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,1,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,0,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,1,S


4 Name Name aways show the Title of one person. and the title can show someinfo 
1 很多参数 都是用的 list [] 作为参数, 里面估计是利用 ndarray 作为参数使用的, 所以 一定要注意 很多时候 [] 做参数没什么大不了的, 这是python的内建类型.
? 2 DataFrame 的访问
3 Series.replace 操作  ([target_list], replace_value). target_list 是一个 待替换的值的 list.
4 DataFrame 的 loc 访问方式.

In [133]:
for df in combine:
    df['Title'] = df.Name.str.extract('([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Jonkheer', 'Don', 'Sir'], 'Royalty')
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Dona'], 'Royalty')
    df['Title'] = df['Title'].replace(['Mme'], 'Mrs')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Rev', 'Major'], 'Officer')
    
    df.loc[(df.Sex == 'male') & (df.Title == 'Dr'), 'Title'] = 'Mr'
    df.loc[(df.Sex == 'female') & (df.Title == 'Dr'), 'Title'] = 'Mrs'
title_group = train_df[['Title', 'Survived']].groupby(['Title'], as_index=False)
title_group_survived_mean = title_group.mean().sort_values(by = 'Survived', ascending=False)
print(title_group_survived_mean)
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royalty": 5, "Officer": 6}
for df in combine:
    df['Title'] = df['Title'].map(title_mapping)
    df['Title'] = df['Title'].fillna(0)
train_df.head()

     Title  Survived
3      Mrs  0.795276
1     Miss  0.702703
5  Royalty  0.600000
0   Master  0.575000
4  Officer  0.181818
2       Mr  0.158700


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,1,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,0,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,1,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,0,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,1,S,1


In [134]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,1,S,1
1,1,1,female,38.0,1,0,71.2833,0,C,3
2,1,3,female,26.0,0,0,7.9250,1,S,2
3,1,1,female,35.0,1,0,53.1000,0,S,3
4,0,3,male,35.0,0,0,8.0500,1,S,1


6 Age 年龄 肯定是有很大影响

In [135]:
sex_mapping = {'male': 0, 'female': 1, 'Child': 2}
for df in combine:
    df.loc[(df.Age < 16), 'Sex'] = 'Child'
    df['Sex'] = df['Sex'].map(sex_mapping).astype(int)

for df in combine:
    df['Age'] = df.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.mean())).astype(int)

train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
age_group = train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index = False)
age_group_survived_mean = age_group.mean().sort_values('Survived', ascending = False)
print(age_group_survived_mean)

         AgeBand  Survived
0  (-0.08, 16.0]  0.550000
3   (48.0, 64.0]  0.434783
2   (32.0, 48.0]  0.404444
1   (16.0, 32.0]  0.339506
4   (64.0, 80.0]  0.090909


In [136]:
for df in combine:
    df.loc[ df['Age'] <= 16.0, 'Age'] = 0
    df.loc[(df['Age'] > 16.0) & (df['Age'] <= 32.0), 'Age'] = 1
    df.loc[(df['Age'] > 32.0) & (df['Age'] <= 48.0), 'Age'] = 2
    df.loc[(df['Age'] > 48.0) & (df['Age'] <= 64.0), 'Age'] = 3
    df.loc[ df['Age'] > 64.0, 'Age'] = 4
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()
    

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,0,1,1,0,7.2500,1,S,1
1,1,1,1,2,1,0,71.2833,0,C,3
2,1,3,1,1,0,0,7.9250,1,S,2
3,1,1,1,2,1,0,53.1000,0,S,3
4,0,3,0,2,0,0,8.0500,1,S,1


7 SibSp and Parch FamilySize
整合 SibSp Parch 两个特征为 FamilySize , 这两个特征主要的作用就是体现亲属关系.

In [137]:
for df in combine:
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

familySize_group = train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False)
familySize_group_survived_mean = familySize_group.mean().sort_values(by='Survived', ascending=False)
print(familySize_group_survived_mean)

   FamilySize  Survived
3           4  0.724138
2           3  0.578431
1           2  0.552795
6           7  0.333333
0           1  0.303538
4           5  0.200000
5           6  0.136364
7           8  0.000000
8          11  0.000000


In [138]:
for df in combine:
    df['IsAlone'] = 0  #more relations
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1  #relations
    df.loc[df['FamilySize'] > 4, 'IsAlone'] = 2   #little relations
train_df = train_df.drop(['SibSp', 'Parch', 'FamilySize'], axis = 1)
test_df = test_df.drop(['SibSp', 'Parch', 'FamilySize'], axis = 1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,IsAlone
0,0,3,0,1,7.2500,1,S,1,0
1,1,1,1,2,71.2833,0,C,3,0
2,1,3,1,1,7.9250,1,S,2,1
3,1,1,1,2,53.1000,0,S,3,0
4,0,3,0,2,8.0500,1,S,1,1


8 创建一个新的feature  --- Age x Pclass
两个重要的特征, 相乘 不知道是否有效果.

In [139]:
for df in combine:
    df['Age*Pclass'] = df['Age']*df['Pclass']
train_df.loc[:, ['Age*Pclass', 'Age', 'Pclass']].head()

,Age*Pclass,Age,Pclass
0,3,1,3
1,2,2,1
2,3,1,3
3,2,2,1
4,6,2,3


9 Embarked 登船港口, fillna 填充默认值.

In [140]:
freq_port = 'S'
for df in combine:
    df['Embarked'] = df['Embarked'].fillna(freq_port)
embarked_group = train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False)
embarked_group_survived_mean = embarked_group.mean()
print(embarked_group_survived_mean)
for df in combine:
    df[]

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


10 Fare 票价, 感觉也是没什么作用的
? 1 Series.dropna 去掉nan 值, 
2 Series.media() 取中间值
3 qcut 这个切分方法 不了解 切分为多少分作为区间范围, 增加Feature FareBand 经常使用 band index作为特征, 感觉这个操作或多或少的减少了一些噪声, 更加抽象于一些概念, 例如救人时先救儿童, 但是并不是救助岁数小, 越好 当大于16岁后 差不多都是一同对待了, 所以 还是要看到band 区域的更符合 概念抽象性.


In [141]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace = True)
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
fare_group = train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False)
fare_group_survived_mean = fare_group.mean().sort_values(by = 'Survived', ascending=True)
print(fare_group_survived_mean)

          FareBand  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


In [142]:
for df in combine:
    df.loc[df['Fare'] <= 7.91, 'Fare'] = 0
    df.loc[(df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
    df.loc[(df['Fare'] > 14.454) & (df['Fare'] <= 31.0), 'Fare'] = 2
    df.loc[(df['Fare'] > 31.0), 'Fare'] = 3
    df['Fare'] = df['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

In [143]:
X_data = train_df.drop(['Survived'], axis=1)
Y_data = train_df['Survived']
X_test = test_df.drop(['PassengerId'], axis=1).copy()
X_test.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,IsAlone,Age*Pclass
0,3,0,2,0,1,Q,1,1,6
1,3,1,2,0,1,S,3,0,6
2,2,0,3,1,1,Q,1,1,6
3,3,0,1,1,1,S,1,1,3
4,3,1,1,1,1,S,3,0,3
